In [5]:
import os
import json
import pandas as pd

In [ ]:
def qwen_organ_annotate(root_path='/download/ct_rate/dataset/train/'):
    desc_info = json.load(open('./metafiles/desc_info.json'))
    conc_info = json.load(open('./metafiles/conc_info.json'))
    organs = ['lung', 'heart', 'esophagus', 'aorta']

    annotation = {}
    patient_ids = sorted(os.listdir(root_path))

    for patient_id in patient_ids:
        sessions = sorted(os.listdir(os.path.join(root_path, patient_id)))
        
        for session in sessions:
            session_info = {}
            recons = sorted(os.listdir(os.path.join(root_path, patient_id, session)))
            session_info['recons'] = recons
            report_list = []
            
            for organ in organs:
                desc = ''
                if organ in desc_info.get(session, {}):
                    desc += desc_info[session][organ]
                    if not desc.endswith('.'):
                        desc += '.'
                if not desc:
                    desc = 'No significant abnormalities.'
                
                conc = ''
                if organ in conc_info.get(session, {}):
                    conc += conc_info[session][organ]
                    if not conc.endswith('.'):
                        conc += '.'
                if not conc:
                    conc = 'No significant abnormalities.'

                report = f'The {organ} shows: {conc} The {organ} looks like: {desc}'
                report = report.replace('"', '')
                report = report.replace('\'', '')  
                report = report.replace('(', '')  
                report = report.replace(')', '')

                report_list.append(report)

            session_info['report'] = report_list
            annotation[session] = session_info
            
    os.makedirs('./files/', exist_ok=True)
    with open(os.path.join('./files/', 'qwen_organ_annotation.json'), 'w') as file:
        json.dump(annotation, file, indent=4)
    return  


def raw_annotate(csv_file='./metafiles/train_reports.csv'):
    df = pd.read_csv(csv_file)
    
    annotation = {}
    
    for index, row in df.iterrows():
        name = str(row['VolumeName'])
        session = name.rsplit('_', 1)[0]
        if session in annotation.keys():
            annotation[session]['recons'].append(name)
        else:
            session_info = {}
            session_info['recons'] = [name]
            impression = str(row['Impressions_EN'])
            if not impression.endswith('.'):
                impression += '.'
            findings = str(row['Findings_EN'])
            if not findings.endswith('.'):
                findings += '.'

            report = f'The volume shows: {impression} The volume looks like: {findings}'
            report = report.replace('"', '')
            report = report.replace('\'', '')  
            report = report.replace('(', '')  
            report = report.replace(')', '')
            
            report_list = [report]
            session_info['report'] = report_list
            annotation[session] = session_info
            
    os.makedirs('./files/', exist_ok=True)
    with open(os.path.join('./files/', 'raw_annotation.json'), 'w') as file:
        json.dump(annotation, file, indent=4)
    
    return
    


In [15]:
qwen_organ_annotate()

In [23]:
raw_annotate()